In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [ ]:
SELECT
md.chembl_id as compound_id,
ass.chembl_id as assay_id,
md.molregno as compound_molregno,
td.pref_name,
ass.assay_organism,
td.organism as td_organism,
ass.relationship_type,
act.standard_relation,
act.standard_value,
act.standard_units,
act.standard_type,
act.activity_comment,
act.doc_id,
td.tid,
tt.parent_type,
tt.target_type,
ass.confidence_score,
cs.canonical_smiles as smiles
FROM target_dictionary td
JOIN target_type tt ON tt.target_type = td.target_type
JOIN assays ass ON td.tid = ass.tid
JOIN activities act ON ass.assay_id = act.assay_id
JOIN molecule_dictionary md ON act.molregno = md.molregno
JOIN compound_structures cs ON md.molregno = cs.molregno
WHERE td.organism = 'Homo sapiens' AND td.pref_name = 'Cytochrome P450 3A4' AND ass.relationship_type = 'D'

In [1]:
import mysql.connector
import json
import pandas as pd

In [2]:
USER = 'tcrd'
HOST = '129.108.3.163'
PASS = 'tcrd'
DB = 'chembl25'

conn = mysql.connector.connect (user=USER, password=PASS,
                               host=HOST,database=DB,buffered=True)
cursor = conn.cursor()

In [3]:
sql = 'Select * from target_dictionary'
df_td = pd.read_sql(sql, conn)

In [4]:
df_td.head()


,tid,target_type,pref_name,tax_id,organism,chembl_id,species_group_flag
0,1,SINGLE PROTEIN,Maltase-glucoamylase,9606.0,Homo sapiens,CHEMBL2074,0
1,2,SINGLE PROTEIN,Sulfonylurea receptor 2,9606.0,Homo sapiens,CHEMBL1971,0
2,3,SINGLE PROTEIN,Phosphodiesterase 5A,9606.0,Homo sapiens,CHEMBL1827,0
3,4,SINGLE PROTEIN,Voltage-gated T-type calcium channel alpha-1H ...,9606.0,Homo sapiens,CHEMBL1859,0
4,5,SINGLE PROTEIN,Nicotinic acetylcholine receptor alpha subunit,6253.0,Ascaris suum,CHEMBL1884,0


In [5]:
df_td['organism'].unique()

array(['Homo sapiens', 'Ascaris suum', 'Escherichia coli K-12', ...,
       'Staphylococcus aureus (strain USA300)',
       'Staphylococcus aureus (strain Mu50 / ATCC 700699)',
       'Pseudomonas putida (strain ATCC 47054 / DSM 6125 / NCIMB 11950 /KT2440)'],
      dtype=object)

In [6]:
len(df_td['organism'].unique())

2494

In [7]:
len(df_td['pref_name'].unique())

9882

In [8]:
drop_index = []
for i in range(len(df_td)):
    if df_td['organism'][i] == 'Homo sapiens':
        continue
    else:
        drop_index.append(i)

In [9]:
#drop_index

In [10]:
df_td.drop(drop_index,axis=0,inplace=True)

In [11]:
df_td.reset_index(drop=True,inplace=True)

In [12]:
len(df_td['organism'].unique())

1

In [13]:
len(df_td['pref_name'].unique())

5066

In [14]:
drop_index = []
for i in range(len(df_td)):
    if 'Cytochrome' in df_td['pref_name'][i] or 'cytochrome' in df_td['pref_name'][i]:
        continue
    else:
        drop_index.append(i)

In [15]:
df_td.drop(drop_index,axis=0,inplace=True)

df_td.reset_index(drop=True,inplace=True)

In [16]:
len(df_td['pref_name'].unique())

47

In [17]:
homo_cyt_list = list(df_td['pref_name'].unique())

In [18]:
from tqdm import tqdm_notebook

In [19]:
for i in tqdm_notebook(range(len(homo_cyt_list))):
    sql = ("""\
        SELECT md.chembl_id as compound_id,
        ass.chembl_id as assay_id,
        md.molregno as compound_molregno,
        td.pref_name,
        ass.assay_organism,
        td.organism as td_organism,
        ass.relationship_type,
        act.standard_relation,
        act.standard_value,
        act.standard_units,
        act.standard_type,
        act.activity_comment,
        act.doc_id,
        td.tid,
        tt.parent_type,
        tt.target_type,
        ass.confidence_score,
        cs.canonical_smiles as smiles
        FROM target_dictionary td
        JOIN target_type tt ON tt.target_type = td.target_type
        JOIN assays ass ON td.tid = ass.tid
        JOIN activities act ON ass.assay_id = act.assay_id
        JOIN molecule_dictionary md ON act.molregno = md.molregno
        JOIN compound_structures cs ON md.molregno = cs.molregno
        WHERE td.organism = 'Homo sapiens' AND td.pref_name = '%s'""" % homo_cyt_list[i]
          )
    conn = mysql.connector.connect (user=USER, password=PASS,
                                   host=HOST,database=DB,buffered=True)
    cursor = conn.cursor()
    
    df = pd.read_sql_query(sql, conn)
  
    homo_cyt_list[i] = homo_cyt_list[i].replace(',', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('/', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace(' ', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace(')', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('(', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('-', '_')
    df.to_csv('../dataset/cyt_homo_data/sql_data/all_relation/'+homo_cyt_list[i]+'.csv')

In [ ]:
  homo_cyt_list[i] = homo_cyt_list[i].replace(',', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('/', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace(' ', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace(')', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('(', '_')
    homo_cyt_list[i] = homo_cyt_list[i].replace('-', '_')

In [86]:
homo_cyt_list[i].replace('/', '_')

'Cytochrome P450 2D6'

In [84]:
homo_cyt_list[i]

'Cytochrome P450 3A4/3A5'

In [90]:
from glob import glob

In [91]:
for i in glob('../dataset/cyt_homo_data/sql_data/*csv'):
    print(i)

../dataset/cyt_homo_data/sql_data/Cytochrome_P450_21.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_51.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_1A1.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_2A6.csv
../dataset/cyt_homo_data/sql_data/NADH_cytochrome_b5_reductase.csv
../dataset/cyt_homo_data/sql_data/NADPH__cytochrome_P450_reductase.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_11B1.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_b_245_heavy_chain.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_4A11.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_c_oxidase_subunit_1.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_26B1.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_3A43.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_2A13.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_2J2.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_P450_4F12.csv
../dataset/cyt_homo_data/sql_data/Cytochrome_b_c1_complex_subunit_7.csv
../dataset/c